In [57]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../deep-neural-networks/notebooks/', one_hot=True, reshape=False)

Extracting ../deep-neural-networks/notebooks/train-images-idx3-ubyte.gz
Extracting ../deep-neural-networks/notebooks/train-labels-idx1-ubyte.gz
Extracting ../deep-neural-networks/notebooks/t10k-images-idx3-ubyte.gz
Extracting ../deep-neural-networks/notebooks/t10k-labels-idx1-ubyte.gz


In [58]:
import tensorflow as tf

# parameters
learning_rate = 0.00001
epochs = 10
batch_size = 128

# number of samples to calculate validation and accuracy
test_valid_size = 256

# network parameters 
n_classes = 10
dropout = 0.75 # prob to keep unit

In [59]:
# weights and biases
weights = {
    'wc1': tf.Variable(tf.random_normal([5, 5, 1, 32])),
    'wc2': tf.Variable(tf.random_normal([5, 5, 32, 64])),
    'wd1': tf.Variable(tf.random_normal([7*7*64, 1024])),
    'out': tf.Variable(tf.random_normal([1024, n_classes]))
}
biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([1024])),
    'out': tf.Variable(tf.random_normal([n_classes])),
}

In [60]:
# 2d convolution layer
def conv2d(x, W, b, strides=1):
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1] , padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# define maxpool function
def maxpool2d(x, k=2):
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1], padding='SAME')

In [61]:
def conv_net(x, weights, biases, dropout):
    # Layer 1 - 28*28*1 to 14*14*32
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    conv1 = maxpool2d(conv1, k=2)

    # Layer 2 - 14*14*32 to 7*7*64
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    conv2 = maxpool2d(conv2, k=2)

    # Fully connected layer - 7*7*64 to 1024
    fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output Layer - class prediction - 1024 to 10
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [62]:
# tf Graph input
x = tf.placeholder(tf.float32, [None, 28, 28, 1])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32)

# Model
logits = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
cost = tf.reduce_mean(\
    tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)\
    .minimize(cost)
    
# Accuracy 
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [70]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch in range(int(mnist.train.num_examples/batch_size)):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: batch_x, y:batch_y, keep_prob: dropout})
            
            # calculate batch loss and accuracy
            loss = sess.run(cost, feed_dict={x: batch_x, y:batch_y, keep_prob: dropout})
            valid_acc = sess.run(accuracy, feed_dict={x: mnist.validation.images[:test_valid_size], 
                                                      y: mnist.validation.labels[:test_valid_size], 
                                                      keep_prob: 1.})
            print('Epoch {:>2}, Batch {:>3}, '
                  'Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(
                epoch + 1,
                batch + 1,
                loss,
                valid_acc))
            
    # Calculate test accuracy
    test_acc = sess.run(accuracy, feed_dict={x: mnist.validation.images[:test_valid_size], 
                                             y: mnist.validation.labels[:test_valid_size], 
                                             keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_acc))
            

Epoch  1, Batch   1, Loss: 93406.6328 Validation Accuracy: 0.074219
Epoch  1, Batch   2, Loss: 80436.0156 Validation Accuracy: 0.093750
Epoch  1, Batch   3, Loss: 72600.3203 Validation Accuracy: 0.121094
Epoch  1, Batch   4, Loss: 64895.0078 Validation Accuracy: 0.136719
Epoch  1, Batch   5, Loss: 59400.4766 Validation Accuracy: 0.152344
Epoch  1, Batch   6, Loss: 56754.5078 Validation Accuracy: 0.156250
Epoch  1, Batch   7, Loss: 54428.8203 Validation Accuracy: 0.175781
Epoch  1, Batch   8, Loss: 55951.4062 Validation Accuracy: 0.171875
Epoch  1, Batch   9, Loss: 49912.0781 Validation Accuracy: 0.210938
Epoch  1, Batch  10, Loss: 48300.8281 Validation Accuracy: 0.218750
Epoch  1, Batch  11, Loss: 49537.8867 Validation Accuracy: 0.234375
Epoch  1, Batch  12, Loss: 44640.0391 Validation Accuracy: 0.242188
Epoch  1, Batch  13, Loss: 46392.6016 Validation Accuracy: 0.250000
Epoch  1, Batch  14, Loss: 45381.0547 Validation Accuracy: 0.234375
Epoch  1, Batch  15, Loss: 47568.4648 Validation

Epoch  1, Batch 122, Loss: 11340.7686 Validation Accuracy: 0.683594
Epoch  1, Batch 123, Loss: 13005.3047 Validation Accuracy: 0.679688
Epoch  1, Batch 124, Loss: 13253.5400 Validation Accuracy: 0.675781
Epoch  1, Batch 125, Loss: 11879.0244 Validation Accuracy: 0.679688
Epoch  1, Batch 126, Loss: 11187.8975 Validation Accuracy: 0.671875
Epoch  1, Batch 127, Loss: 12545.5146 Validation Accuracy: 0.679688
Epoch  1, Batch 128, Loss: 11663.4922 Validation Accuracy: 0.687500
Epoch  1, Batch 129, Loss: 11197.4424 Validation Accuracy: 0.687500
Epoch  1, Batch 130, Loss: 10528.6865 Validation Accuracy: 0.691406
Epoch  1, Batch 131, Loss: 11540.4688 Validation Accuracy: 0.687500
Epoch  1, Batch 132, Loss: 11381.0352 Validation Accuracy: 0.691406
Epoch  1, Batch 133, Loss:  8888.5312 Validation Accuracy: 0.683594
Epoch  1, Batch 134, Loss: 12986.5518 Validation Accuracy: 0.683594
Epoch  1, Batch 135, Loss:  8117.6460 Validation Accuracy: 0.679688
Epoch  1, Batch 136, Loss:  9917.6660 Validation

Epoch  1, Batch 243, Loss:  7239.3647 Validation Accuracy: 0.726562
Epoch  1, Batch 244, Loss:  6835.4355 Validation Accuracy: 0.730469
Epoch  1, Batch 245, Loss:  7588.3594 Validation Accuracy: 0.718750
Epoch  1, Batch 246, Loss:  6657.9150 Validation Accuracy: 0.730469
Epoch  1, Batch 247, Loss:  5598.4834 Validation Accuracy: 0.730469
Epoch  1, Batch 248, Loss:  6315.1313 Validation Accuracy: 0.734375
Epoch  1, Batch 249, Loss:  6374.4287 Validation Accuracy: 0.746094
Epoch  1, Batch 250, Loss:  5424.3994 Validation Accuracy: 0.742188
Epoch  1, Batch 251, Loss:  6973.7407 Validation Accuracy: 0.750000
Epoch  1, Batch 252, Loss:  5307.7749 Validation Accuracy: 0.750000
Epoch  1, Batch 253, Loss:  5676.3193 Validation Accuracy: 0.746094
Epoch  1, Batch 254, Loss:  6606.3535 Validation Accuracy: 0.738281
Epoch  1, Batch 255, Loss:  6695.9487 Validation Accuracy: 0.738281
Epoch  1, Batch 256, Loss:  6305.4316 Validation Accuracy: 0.734375
Epoch  1, Batch 257, Loss:  6933.5117 Validation

Epoch  1, Batch 364, Loss:  5810.3096 Validation Accuracy: 0.773438
Epoch  1, Batch 365, Loss:  4705.8408 Validation Accuracy: 0.773438
Epoch  1, Batch 366, Loss:  4922.1895 Validation Accuracy: 0.765625
Epoch  1, Batch 367, Loss:  4275.0752 Validation Accuracy: 0.773438
Epoch  1, Batch 368, Loss:  3829.3813 Validation Accuracy: 0.777344
Epoch  1, Batch 369, Loss:  3979.0156 Validation Accuracy: 0.777344
Epoch  1, Batch 370, Loss:  4827.3755 Validation Accuracy: 0.781250
Epoch  1, Batch 371, Loss:  4218.9561 Validation Accuracy: 0.785156
Epoch  1, Batch 372, Loss:  4460.0244 Validation Accuracy: 0.785156
Epoch  1, Batch 373, Loss:  5360.8696 Validation Accuracy: 0.785156
Epoch  1, Batch 374, Loss:  4833.8843 Validation Accuracy: 0.785156
Epoch  1, Batch 375, Loss:  5124.3848 Validation Accuracy: 0.785156
Epoch  1, Batch 376, Loss:  5032.6934 Validation Accuracy: 0.785156
Epoch  1, Batch 377, Loss:  4737.5278 Validation Accuracy: 0.785156
Epoch  1, Batch 378, Loss:  4400.9692 Validation

Epoch  2, Batch  56, Loss:  4210.8433 Validation Accuracy: 0.789062
Epoch  2, Batch  57, Loss:  4211.8301 Validation Accuracy: 0.789062
Epoch  2, Batch  58, Loss:  3588.2412 Validation Accuracy: 0.785156
Epoch  2, Batch  59, Loss:  4225.8994 Validation Accuracy: 0.781250
Epoch  2, Batch  60, Loss:  3833.9282 Validation Accuracy: 0.781250
Epoch  2, Batch  61, Loss:  3924.6746 Validation Accuracy: 0.789062
Epoch  2, Batch  62, Loss:  3063.3794 Validation Accuracy: 0.789062
Epoch  2, Batch  63, Loss:  3291.9854 Validation Accuracy: 0.789062
Epoch  2, Batch  64, Loss:  4079.0530 Validation Accuracy: 0.789062
Epoch  2, Batch  65, Loss:  3859.3394 Validation Accuracy: 0.785156
Epoch  2, Batch  66, Loss:  4102.5342 Validation Accuracy: 0.789062
Epoch  2, Batch  67, Loss:  3540.6943 Validation Accuracy: 0.792969
Epoch  2, Batch  68, Loss:  3297.9207 Validation Accuracy: 0.785156
Epoch  2, Batch  69, Loss:  3060.9963 Validation Accuracy: 0.789062
Epoch  2, Batch  70, Loss:  3489.3796 Validation

...

Epoch  6, Batch 276, Loss:   937.3491 Validation Accuracy: 0.808594
Epoch  6, Batch 277, Loss:  1003.9338 Validation Accuracy: 0.804688
Epoch  6, Batch 278, Loss:   767.9042 Validation Accuracy: 0.808594
Epoch  6, Batch 279, Loss:   941.0607 Validation Accuracy: 0.808594
Epoch  6, Batch 280, Loss:  1125.6814 Validation Accuracy: 0.808594
Epoch  6, Batch 281, Loss:   839.5315 Validation Accuracy: 0.804688
Epoch  6, Batch 282, Loss:   871.0339 Validation Accuracy: 0.804688
Epoch  6, Batch 283, Loss:   989.0004 Validation Accuracy: 0.808594
Epoch  6, Batch 284, Loss:   857.3191 Validation Accuracy: 0.812500
Epoch  6, Batch 285, Loss:   854.4085 Validation Accuracy: 0.808594
Epoch  6, Batch 286, Loss:   852.7577 Validation Accuracy: 0.812500
Epoch  6, Batch 287, Loss:   764.3488 Validation Accuracy: 0.804688
Epoch  6, Batch 288, Loss:   908.9784 Validation Accuracy: 0.812500
Epoch  6, Batch 289, Loss:   790.1818 Validation Accuracy: 0.812500
Epoch  6, Batch 290, Loss:  1110.3643 Validation

Epoch  6, Batch 397, Loss:   895.7556 Validation Accuracy: 0.796875
Epoch  6, Batch 398, Loss:   794.0327 Validation Accuracy: 0.792969
Epoch  6, Batch 399, Loss:   803.3309 Validation Accuracy: 0.796875
Epoch  6, Batch 400, Loss:   588.0583 Validation Accuracy: 0.800781
Epoch  6, Batch 401, Loss:   880.4542 Validation Accuracy: 0.804688
Epoch  6, Batch 402, Loss:   744.4480 Validation Accuracy: 0.800781
Epoch  6, Batch 403, Loss:   888.9417 Validation Accuracy: 0.808594
Epoch  6, Batch 404, Loss:   736.9949 Validation Accuracy: 0.804688
Epoch  6, Batch 405, Loss:   608.1919 Validation Accuracy: 0.808594
Epoch  6, Batch 406, Loss:   629.3982 Validation Accuracy: 0.804688
Epoch  6, Batch 407, Loss:   985.3642 Validation Accuracy: 0.804688
Epoch  6, Batch 408, Loss:   981.9509 Validation Accuracy: 0.804688
Epoch  6, Batch 409, Loss:   967.6820 Validation Accuracy: 0.808594
Epoch  6, Batch 410, Loss:   747.4050 Validation Accuracy: 0.804688
Epoch  6, Batch 411, Loss:   970.7407 Validation

Epoch  7, Batch  89, Loss:   935.0864 Validation Accuracy: 0.804688
Epoch  7, Batch  90, Loss:   785.1583 Validation Accuracy: 0.800781
Epoch  7, Batch  91, Loss:   745.3806 Validation Accuracy: 0.812500
Epoch  7, Batch  92, Loss:   682.5601 Validation Accuracy: 0.816406
Epoch  7, Batch  93, Loss:  1001.2546 Validation Accuracy: 0.816406
Epoch  7, Batch  94, Loss:   819.0912 Validation Accuracy: 0.812500
Epoch  7, Batch  95, Loss:   737.6818 Validation Accuracy: 0.812500
Epoch  7, Batch  96, Loss:   932.8203 Validation Accuracy: 0.816406
Epoch  7, Batch  97, Loss:   637.6912 Validation Accuracy: 0.816406
Epoch  7, Batch  98, Loss:   986.6460 Validation Accuracy: 0.816406
Epoch  7, Batch  99, Loss:   877.9945 Validation Accuracy: 0.820312
Epoch  7, Batch 100, Loss:  1003.0532 Validation Accuracy: 0.812500
Epoch  7, Batch 101, Loss:   784.8449 Validation Accuracy: 0.816406
Epoch  7, Batch 102, Loss:   670.9774 Validation Accuracy: 0.816406
Epoch  7, Batch 103, Loss:   717.8473 Validation

Epoch  7, Batch 210, Loss:   684.9951 Validation Accuracy: 0.824219
Epoch  7, Batch 211, Loss:   715.0258 Validation Accuracy: 0.816406
Epoch  7, Batch 212, Loss:   809.4885 Validation Accuracy: 0.816406
Epoch  7, Batch 213, Loss:   680.2567 Validation Accuracy: 0.808594
Epoch  7, Batch 214, Loss:   745.0872 Validation Accuracy: 0.808594
Epoch  7, Batch 215, Loss:   711.9576 Validation Accuracy: 0.820312
Epoch  7, Batch 216, Loss:   769.3218 Validation Accuracy: 0.824219
Epoch  7, Batch 217, Loss:   872.2437 Validation Accuracy: 0.828125
Epoch  7, Batch 218, Loss:   626.2004 Validation Accuracy: 0.828125
Epoch  7, Batch 219, Loss:   815.4139 Validation Accuracy: 0.835938
Epoch  7, Batch 220, Loss:   747.5919 Validation Accuracy: 0.828125
Epoch  7, Batch 221, Loss:   913.2051 Validation Accuracy: 0.824219


KeyboardInterrupt: 

In [56]:
batch_x.shape

(128, 784)